# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

In [81]:
import numpy as np
import pandas as pd

import platform
if platform.system() == "Windows":
    BASE_DIR = r'C:\Users\Alex\Desktop\Academic\UChicago\FINM 37500\finm-fiderivs-2024'
else:
    BASE_DIR = r'/Users/alexhuang/Documents/Academic/FINM 37500/finm-fiderivs-2024'


import sys, os
sys.path.append(BASE_DIR)

from cmds.binomial import *
from cmds.ratecurves import *
from cmds.volskew import *
from cmds.ficcvol import *
from cmds.fi_binomial_model import FIBinomialModel

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

In [113]:

class FIBinomialModel():
    def __init__(self, face_value: float=100) -> None:
        self.__face_value: float = face_value
        self.__rate_tree: pd.DataFrame = None
        self.__bond_tree: pd.DataFrame = None
        self.__swap_tree: pd.DataFrame = None
        self.__cashflow_tree: pd.DataFrame = None
        self.__valuation_tree: pd.DataFrame = None

        self.__dt: float = None
        self.__T: float = None
    
    @property
    def rate_tree(self) -> pd.DataFrame:
        return self.__rate_tree

    @rate_tree.setter
    def rate_tree(self, new_rate_tree: pd.DataFrame) -> None:
        if isinstance(new_rate_tree, pd.DataFrame):
            self.__rate_tree = new_rate_tree
        else:
            raise TypeError("Can only set rate_tree attribute to pd.DataFrame type only.")
        
    @property
    def bond_tree(self) -> pd.DataFrame:
        return self.__bond_tree
    
    @bond_tree.setter
    def bond_tree(self, new_bond_tree: pd.DataFrame) -> None:
        if isinstance(new_bond_tree, pd.DataFrame):
            self.__bond_tree = new_bond_tree
        else:
            raise TypeError("Can only set bond_tree attribute to pd.DataFrame type only.")
    
    @property
    def cashflow_tree(self) -> pd.DataFrame:
        return self.__cashflow_tree

    @property
    def p_star(self):
        if (self.__rate_tree is None or self.__bond_tree is None or self.__dt is None or self.__T is None):
            raise Exception("Compute rate tree and bond tree first.")
        
        A = np.exp(self.__rate_tree.iloc[0,0] * self.__dt)

        pstar = (A * self.__bond_tree.loc[0,0] - self.__bond_tree.loc[1,self.__dt])/(self.__bond_tree.loc[0,self.__dt] - self.__bond_tree.loc[1,self.__dt])
        return pstar

    def initialize_empty_rate_tree(self, dt, T) -> None:
        self.__dt = dt
        self.__T = T
        timegrid = pd.Series((np.arange(0,round(T/dt)+1)*dt).round(6),name='time',index=pd.Index(range(round(T/dt)+1),name='state'))
        tree = pd.DataFrame(dtype=float,columns=timegrid,index=timegrid.index)
        self.__rate_tree = tree

    def compute_bond_tree(self, t0_price: float=None) -> None:
        maturity = (self.__rate_tree.columns[-1] - self.__rate_tree.columns[-2]) + self.__rate_tree.columns[-1]
        bond_tree_cols = list(self.__rate_tree.columns.copy().values)
        bond_tree_cols.append(maturity)
        print(bond_tree_cols)
        bond_tree = pd.DataFrame(dtype=float, index=pd.RangeIndex(0, len(bond_tree_cols)), columns=bond_tree_cols)
        bond_tree.index.name = self.__rate_tree.index.name
        bond_tree.columns.name = self.__rate_tree.columns.name
        
        forward_time: float = bond_tree.columns[-1]
        for time in reversed(bond_tree.columns):
            if time == maturity:
                bond_tree[time] = float(self.__face_value)
                forward_time = time
                continue
            bond_tree[time] = np.exp(-self.__rate_tree[time]*(forward_time-time)) * ((bond_tree[forward_time] + bond_tree[forward_time].shift(-1)) / 2)
            forward_time = time
        if t0_price:
            bond_tree.iloc[0,0] = t0_price
        self.__bond_tree = bond_tree

    def compute_swap_tree(self, payoff_func) -> pd.DataFrame:
        Z = np.exp(-self.__rate_tree.iloc[0,0] * self.__dt)
        swap_tree = pd.DataFrame(index=self.__rate_tree.index, columns=self.__rate_tree.columns, dtype=float)
        swap_tree[self.__dt] = payoff_func(self.__rate_tree[self.__dt])
        swap_tree.loc[0,0] = Z * np.array([self.p_star,1-self.p_star])@ swap_tree[self.__dt].values
        self.__swap_tree = swap_tree
        return swap_tree

    def construct_bond_cftree(self, T, compound, cpn, cpn_freq=2, face=100) -> pd.DataFrame:
        step = int(compound/cpn_freq)

        self.initialize_empty_rate_tree(1/compound, T)
        cftree = self.__rate_tree.copy()
        cftree.iloc[:,:] = 0
        cftree.iloc[:, -1:0:-step] = (cpn/cpn_freq) * face
        
        # final cashflow is accounted for in payoff function
        # drop final period cashflow from cashflow tree
        cftree = cftree.iloc[:-1,:-1]
        
        self.__cashflow_tree = cftree
        return cftree

    def bintree_pricing(self, payoff=None, ratetree=None, undertree=None,cftree=None, dt=None, pstars=None, timing=None, cfdelay=False,style='european',Tamerican=0):
    
        if payoff is None:
            payoff = lambda r: 0
        
        if undertree is None:
            undertree = ratetree
            
        if cftree is None:
            cftree = pd.DataFrame(0, index=undertree.index, columns=undertree.columns)
            
        if pstars is None:
            pstars = pd.Series(.5, index=undertree.columns)

        if dt is None:
            dt = undertree.columns.to_series().diff().mean()
            dt = undertree.columns[1]-undertree.columns[0]
        
        if timing == 'deferred':
            cfdelay = True
        
        if dt<.25 and cfdelay:
            display('Warning: cfdelay setting only delays by dt.')
            
        valuetree = pd.DataFrame(dtype=float, index=undertree.index, columns=undertree.columns)

        for steps_back, t in enumerate(valuetree.columns[-1::-1]):
            if steps_back==0:                           
                valuetree[t] = payoff(undertree[t])
                if cfdelay:
                    valuetree[t] *= np.exp(-ratetree[t]*dt)
            else:
                for state in valuetree[t].index[:-1]:
                    val_avg = pstars[t] * valuetree.iloc[state,-steps_back] + (1-pstars[t]) * valuetree.iloc[state+1,-steps_back]
                    
                    if cfdelay:
                        cf = cftree.loc[state,t]
                    else:                    
                        cf = cftree.iloc[state,-steps_back]
                    
                    valuetree.loc[state,t] = np.exp(-ratetree.loc[state,t]*dt) * (val_avg + cf)

                if style=='american':
                    if t>= Tamerican:
                        valuetree.loc[:,t] = np.maximum(valuetree.loc[:,t],payoff(undertree.loc[:,t]))
            
        return valuetree


    def display_rate_tree(self) -> pd.DataFrame:
        return self.__rate_tree.style.format('{:.4%}',na_rep='').format_index('{:.2f}',axis=1)
    
    def display_bond_tree(self) -> pd.DataFrame:
        return self.__bond_tree.style.format('$ {:.4}',na_rep='').format_index('{:.2f}',axis=1)

In [82]:
cap_curves_df = pd.read_excel(r'../data/cap_curves_2024-02-20.xlsx', index_col=0)
cap_curves_df.head(5)

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247


In [83]:
cap_df = pd.read_excel(r'../data/cap_quotes_2024-02-20.xlsx')
cap_df.head(5)

,date,USCNSQ1 SMKO Curncy,USCNSQ2 SMKO Curncy,USCNSQ3 SMKO Curncy,USCNSQ4 SMKO Curncy,USCNSQ5 SMKO Curncy,USCNSQ6 SMKO Curncy,USCNSQ7 SMKO Curncy,USCNSQ8 SMKO Curncy,USCNSQ9 SMKO Curncy,USCNSQ10 SMKO Curncy
0,maturity,1.002053,2.001369,3.000684,4.0,5.002053,6.001369,7.000684,8.0,9.002053,10.001369
1,2022-03-17 00:00:00,127.300000,108.500000,109.900000,108.5,107.300000,104.300000,101.000000,97.7,95.200000,93.200000
2,2022-03-18 00:00:00,96.200000,104.900000,108.000000,108.5,108.000000,105.000000,101.800000,98.3,95.400000,93.400000
3,2022-03-21 00:00:00,96.200000,105.000000,108.100000,108.5,107.900000,105.000000,101.700000,98.2,95.400000,93.400000
4,2022-03-22 00:00:00,75.100000,108.300000,115.200000,115.3,113.600000,109.900000,106.300000,102.8,99.900000,97.500000


In [84]:
swap_vol_df = pd.read_excel(r'../data/swaption_vol_data.xlsx')
swap_vol_df

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

In [85]:
swaption_exp = swap_vol_df.loc[0, 'expiration']
swaption_end = swaption_exp + swap_vol_df.loc[0, 'tenor'] 
fwd_swap_rate = calc_fwdswaprate(cap_curves_df['discounts'], swaption_exp, swaption_end, freqswap=4)
fwd_swap_rate

0.03672212061985555


## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

In [86]:
swaption_implied_df = pd.DataFrame(index=swap_vol_df.columns[-9:], columns=['IV', 'strikes', 'prices'])
swaption_implied_df['IV'] = swap_vol_df[swap_vol_df.columns[-9:]].transpose() / 100
swaption_implied_df['strikes'] = (fwd_swap_rate + swap_vol_df.columns[-9:].astype(int)/10000)
discount = cap_curves_df['discounts'].loc[1.25:5].sum()/4
# print(swaption_exp, swaption_implied_df['IV'].values, swaption_implied_df['strikes'].values, fwd_swap_rate, discount, sep='\n')
swaption_implied_df['prices'] = blacks_formula(swaption_exp, swaption_implied_df['IV'].values, swaption_implied_df['strikes'].values, fwd_swap_rate, discount)

display(swaption_implied_df)

,IV,strikes,prices
-200,0.5454,0.016722,0.072031
-100,0.4037,0.026722,0.040653
-50,0.3594,0.031722,0.027398
-25,0.3423,0.034222,0.021749
0,0.3283,0.036722,0.016874
25,0.3171,0.039222,0.012812
50,0.3086,0.041722,0.009556
100,0.2983,0.046722,0.005133
200,0.2954,0.056722,0.001473


## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [87]:
alt_swaps_df = pd.DataFrame(columns=['exp', 'tenor', 'strike', 'prices'], index=pd.RangeIndex(0,3))
alt_swaps_df['exp'] = [0.25, 2, 1]
alt_swaps_df['tenor'] = [4, 4, 2]
alt_swaps_df['strike'] = 'ATM'

def price_swaps(exp, tenor, atm_iv, atm_strike, freq=4):
    print( exp, exp + tenor, sep='\n')
    print()
    fwd_swap_rate = calc_fwdswaprate(cap_curves_df['discounts'], exp, exp + tenor, freqswap=freq)
    discount = cap_curves_df['discounts'].loc[exp+(1/freq):exp+tenor].sum() / freq
    # print(cap_curves_df['discounts'].loc[exp+(1/freq):exp+tenor])
    # print(discount)
    print(exp, atm_iv, atm_strike, fwd_swap_rate, discount, sep='\n')
    price = blacks_formula(exp, atm_iv, atm_strike, fwd_swap_rate, discount) * 100
    return price

# alt_swaps_df['prices'] = 
price_swaps(alt_swaps_df['exp'][0], alt_swaps_df['tenor'][0], swaption_implied_df.loc[0,'IV'], swaption_implied_df.loc[0, 'strikes'])


0.25
4.25

0.25
0.3283
0.03672212061985555
0.03910405311265784
3.623807757672832


1.3951918634840146

***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

In [88]:
theta, rate_tree = estimate_theta(cap_curves_df['fwd vols'].bfill(), cap_curves_df['discounts']) 
simga = cap_curves_df['fwd vols']

In [89]:
theta.to_frame().transpose()

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
theta,NaN,-0.117846,-0.262526,-0.359865,-0.720504,-0.459257,-0.384915,-0.244081,-0.396789,-0.033089,...,0.125541,0.134603,0.153908,0.169295,0.178789,0.182346,0.179249,0.168698,0.149782,0.121431


In [90]:
# Exp
rate_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
state,,,,,,,,,,,,,,,,,,,,,
0,0.051874,0.054727,0.056943,0.059101,0.059130,0.063777,0.070361,0.080314,0.087783,0.104426,...,4.027980,4.771745,5.651177,6.686142,7.901102,9.325144,10.991845,12.939920,15.213894,17.864737
1,NaN,0.046355,0.048232,0.050060,0.050085,0.054021,0.059598,0.068028,0.074354,0.088451,...,3.411800,4.041788,4.786689,5.663330,6.692432,7.898632,9.310369,10.960437,12.886551,15.131881
2,NaN,NaN,0.039071,0.040551,0.040572,0.043760,0.048278,0.055107,0.060231,0.071651,...,2.763761,3.274089,3.877503,4.587634,5.421267,6.398361,7.541952,8.878606,10.438872,12.257723
3,NaN,NaN,NaN,0.031446,0.031461,0.033934,0.037437,0.042732,0.046706,0.055562,...,2.143158,2.538891,3.006809,3.557480,4.203921,4.961608,5.848405,6.884913,8.094821,9.505248
4,NaN,NaN,NaN,NaN,0.021922,0.023645,0.026086,0.029776,0.032545,0.038716,...,1.493367,1.769117,2.095165,2.478876,2.929321,3.457282,4.075209,4.797454,5.640527,6.623322
5,NaN,NaN,NaN,NaN,NaN,0.016155,0.017823,0.020344,0.022236,0.026452,...,1.020306,1.208706,1.431470,1.693631,2.001386,2.362103,2.784286,3.277743,3.853751,4.525222
6,NaN,NaN,NaN,NaN,NaN,NaN,0.012080,0.013788,0.015071,0.017928,...,0.691529,0.819219,0.970201,1.147885,1.356471,1.600953,1.887094,2.221542,2.611941,3.067041
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009367,0.010238,0.012179,...,0.469779,0.556523,0.659090,0.779797,0.921496,1.087580,1.281966,1.509167,1.774378,2.083543
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007028,0.008360,...,0.322471,0.382015,0.452421,0.535278,0.632545,0.746551,0.879983,1.035942,1.217991,1.430212


In [91]:
# Annual
np.log(rate_tree + 1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
state,,,,,,,,,,,,,,,,,,,,,
0,0.050573,0.053282,0.055381,0.057420,0.057448,0.061826,0.067996,0.077251,0.084142,0.099326,...,1.615018,1.752974,1.894794,2.039419,2.186175,2.334582,2.484227,2.634757,2.785869,2.937294
1,NaN,0.045313,0.047105,0.048847,0.048871,0.052612,0.057889,0.065814,0.071720,0.084756,...,1.484283,1.617761,1.755560,1.896619,2.040237,2.185898,2.333150,2.481604,2.630921,2.780797
2,NaN,NaN,0.038327,0.039751,0.039770,0.042830,0.047148,0.053642,0.058487,0.069200,...,1.325419,1.452571,1.584633,1.720556,1.859616,2.001259,2.144990,2.290371,2.437017,2.584580
3,NaN,NaN,NaN,0.030961,0.030976,0.033371,0.036753,0.041845,0.045649,0.054073,...,1.145228,1.263813,1.387995,1.516770,1.649412,1.785340,1.924016,2.064951,2.207705,2.351875
4,NaN,NaN,NaN,NaN,0.021686,0.023370,0.025752,0.029341,0.032027,0.037985,...,0.913634,1.018528,1.129841,1.246709,1.368467,1.494539,1.624368,1.757419,1.893191,2.031212
5,NaN,NaN,NaN,NaN,NaN,0.016026,0.017666,0.020140,0.021992,0.026108,...,0.703249,0.792407,0.888496,0.990890,1.099074,1.212567,1.330857,1.453425,1.579752,1.709323
6,NaN,NaN,NaN,NaN,NaN,NaN,0.012007,0.013694,0.014958,0.017769,...,0.525633,0.598407,0.678136,0.764484,0.857165,0.955878,1.060250,1.169860,1.284245,1.402916
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009323,0.010186,0.012105,...,0.385112,0.442454,0.506269,0.576499,0.653104,0.736006,0.825037,0.919951,1.020427,1.126079
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007003,0.008325,...,0.279502,0.323543,0.373232,0.428711,0.490140,0.557643,0.631263,0.710958,0.796602,0.887978


## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

In [92]:
fwd_swap_rate

0.03672212061985555

In [119]:
FACE = 100
T = 5
compound = 4
cpn = fwd_swap_rate
cpn_freq = 2

In [120]:
payoff = lambda r: payoff_bond(r, 1/compound, facevalue = FACE * (1 + cpn / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, cpn, cpn_freq=cpn_freq)
bond_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:int(T * compound),:int(T * compound)], cftree=cash_flow_tree)
bond_tree.round(4)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,98.702,97.4898,94.3490,92.8990,89.4852,87.6976,84.0015,82.0899,78.4115,76.5693,73.1477,71.8807,69.2857,68.9351,67.4588,68.8420,69.7294,73.7711,78.1060,87.4781
1,NaN,102.4909,99.6444,98.5044,95.4061,93.9382,90.5405,88.8944,85.4259,83.7332,80.3693,79.0606,76.3051,75.6872,73.8176,74.6827,74.9137,78.1841,81.5362,89.5357
2,NaN,NaN,104.0544,103.2020,100.4115,99.2783,96.2182,94.9047,91.7402,90.3241,87.1694,85.9912,83.2546,82.5488,80.4420,80.8921,80.4924,82.9397,85.2073,91.7519
3,NaN,NaN,NaN,106.9495,104.4234,103.5920,100.8504,99.8682,97.0301,95.9401,93.0716,92.1277,89.5447,88.9156,86.7535,86.9657,86.0906,87.8088,88.9719,93.9256
4,NaN,NaN,NaN,NaN,107.4915,106.9040,104.4265,103.7292,101.1794,100.3992,97.8203,97.1377,94.7572,94.2930,92.2064,92.3455,91.1900,92.4164,92.7247,96.2568
5,NaN,NaN,NaN,NaN,NaN,109.2604,106.9768,106.4904,104.1568,103.6128,101.2589,100.7837,98.5698,98.2484,96.2406,96.3444,94.9928,95.8595,95.5284,97.9903
6,NaN,NaN,NaN,NaN,NaN,NaN,108.7562,108.4186,106.2378,105.8620,103.6694,103.3441,101.2520,101.0372,99.0918,99.1758,97.6885,98.3012,97.5147,99.2134
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.7516,107.6759,107.4156,105.3335,105.1105,103.1010,102.9582,101.0547,101.1232,99.5400,99.9756,98.8736,100.0470
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.6697,108.4881,106.4807,106.3260,104.3706,104.2741,102.3954,102.4492,100.7963,101.1071,99.7878,100.6046


## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

In [122]:
payer_swap_tree = 100 - bond_tree
payer_swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,1.298017,2.510201,5.651032,7.100999,10.514763,12.302403,15.998539,17.910126,21.588536,23.430670,26.852265,28.119317,30.714301,31.064925,32.541225,31.157977,30.270626,26.228940,21.894036,12.521886
1,NaN,-2.490852,0.355579,1.495591,4.593869,6.061836,9.459525,11.105595,14.574140,16.266768,19.630740,20.939393,23.694948,24.312833,26.182436,25.317292,25.086261,21.815863,18.463795,10.464324
2,NaN,NaN,-4.054385,-3.202016,-0.411521,0.721689,3.781849,5.095263,8.259762,9.675920,12.830607,14.008841,16.745375,17.451150,19.557953,19.107870,19.507627,17.060254,14.792729,8.248142
3,NaN,NaN,NaN,-6.949468,-4.423434,-3.592023,-0.850388,0.131774,2.969946,4.059876,6.928394,7.872315,10.455301,11.084371,13.246466,13.034271,13.909433,12.191186,11.028090,6.074383
4,NaN,NaN,NaN,NaN,-7.491460,-6.903965,-4.426549,-3.729171,-1.179424,-0.399229,2.179681,2.862305,5.242770,5.707034,7.793617,7.654452,8.810023,7.583572,7.275270,3.743180
5,NaN,NaN,NaN,NaN,NaN,-9.260424,-6.976793,-6.490424,-4.156797,-3.612797,-1.258875,-0.783720,1.430159,1.751623,3.759422,3.655649,5.007195,4.140491,4.471613,2.009703
6,NaN,NaN,NaN,NaN,NaN,NaN,-8.756178,-8.418601,-6.237811,-5.862027,-3.669406,-3.344127,-1.252011,-1.037154,0.908178,0.824187,2.311537,1.698811,2.485335,0.786582
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.751617,-7.675925,-7.415643,-5.333510,-5.110507,-3.100953,-2.958217,-1.054689,-1.123230,0.459958,0.024436,1.126376,-0.046985
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.669714,-8.488107,-6.480672,-6.326032,-4.370632,-4.274099,-2.395438,-2.449179,-0.796274,-1.107060,0.212160,-0.604586


## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

In [124]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

swaption_tree = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=payer_swap_tree.iloc[:tsteps,:tsteps])
swaption_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.711,2.914,4.787,7.444,10.515
1,NaN,0.554,1.121,2.268,4.594
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

In [126]:
bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=payer_swap_tree.iloc[:tsteps,:tsteps], style='american').round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.922,3.340,5.651,7.444,10.515
1,NaN,0.554,1.121,2.268,4.594
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

## 3.2

Price the **american** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

***